# Week 5 Homework

***Due (pushed to your GitHub branch) on 10/18 by 11:59 pm***

## FASTQ fetch, QC, and trimming/filtering

Use the SRA-toolkit to fetch the relevant FASTQ files. You may use a hand-crafted `txt` file to provide the list of SRAs instead of manipulating an SRA Run Table with `cut`.

In [2]:
# ERR4231022
# ERR4231010
# ERR4231012
# ERR4231013
# ERR4231014
# ERR4231015
# ERR4231016
# ERR4231017
# ERR4231018
# ERR4231019
# ERR4231020
# ERR4231021
# ERR4231023
# ERR4231011
# print("ERR4231022 ERR4231010 ERR4231012 ERR4231013 ERR4231014 ERR4231015 ERR4231016 ERR4231017 ERR4231018 ERR4231019 ERR4231020 ERR4231021 ERR4231023 ERR4231011".split(" "))


['ERR4231022', 'ERR4231010', 'ERR4231012', 'ERR4231013', 'ERR4231014', 'ERR4231015', 'ERR4231016', 'ERR4231017', 'ERR4231018', 'ERR4231019', 'ERR4231020', 'ERR4231021', 'ERR4231023', 'ERR4231011']


Run QC on the FASTQ files and aggregate the results into a report.

In [1]:
# !mkdir fastq_placenta
# !cut -d ',' -f 1 SraRunTable_Placenta.txt | tail -n +2 > sra_list_placenta.txt
!while IFS= read -r line; do \
    echo "Getting $line from NCBI SRA"; \
    parallel-fastq-dump --sra-id $line --threads 16 --outdir fastq_placenta --gzip; \
    done < sra_list_placenta.txt

Getting ERR4231022 from NCBI SRA
2024-10-31 15:35:06,751 - SRR ids: ['ERR4231022']
2024-10-31 15:35:06,751 - extra args: ['--gzip']
2024-10-31 15:35:06,751 - tempdir: /local/scratch/job_102381/pfd_s1fwvzt3
2024-10-31 15:35:06,752 - CMD: sra-stat --meta --quick ERR4231022
2024-10-31 15:35:09,579 - ERR4231022 spots: 55208970
2024-10-31 15:35:09,579 - blocks: [[1, 3450560], [3450561, 6901120], [6901121, 10351680], [10351681, 13802240], [13802241, 17252800], [17252801, 20703360], [20703361, 24153920], [24153921, 27604480], [27604481, 31055040], [31055041, 34505600], [34505601, 37956160], [37956161, 41406720], [41406721, 44857280], [44857281, 48307840], [48307841, 51758400], [51758401, 55208970]]
2024-10-31 15:35:09,579 - CMD: fastq-dump -N 1 -X 3450560 -O /local/scratch/job_102381/pfd_s1fwvzt3/0 --gzip ERR4231022
2024-10-31 15:35:09,581 - CMD: fastq-dump -N 3450561 -X 6901120 -O /local/scratch/job_102381/pfd_s1fwvzt3/1 --gzip ERR4231022
2024-10-31 15:35:09,581 - CMD: fastq-dump -N 6901121 

Edit this block with a short evaluation of each of the report sections. Each evaluation should include your important observations, reasons for potential QC failure, and an opinion on what should be done on the failure (i.e., trimming or filtering). Problem FASTQ files, if any, should be noted.

*Per base sequence quality:*

    Samples ERR42310[19-21] have a low sequence quality after at base 172 with 177 being where the low points start. This could be due to a poly-A tail.
    Samples ERR42310[17&18] drop of in quality at base 187.
    The other samples seem to have decent per base sequence qualities.
*Per tile sequence quality:*

    no report on my multiqc file
*Per sequence quality scores:*

    The Phred scores are within high quality ranges for every sample. 
*Per base sequence content:*

    Before bp 10 there is a low sequence content score, this is likely due to an adapter or identifier. 
*Per sequence GC content:*

    Samples ERR42310[16-18,20,22-23] have warnings for their GC content with some sequences having 61% gc content.
*Per base N content:*

    Samples ERR42310[19-21] have a high percentage of per base N content starting at bp 177 which is similar to their quality drop off points, likely due to the same thing that lowered their quality score.   
*Sequence length distribution:*

    All sequences had a length of 202 bp
*Sequence duplication levels:*

    Samples ERR42310[11&14] have warnings for their duplication levels while the rest of the samples failed their duplication levels. They all had over 10k sequences that were repeated or duplicated
*Overrepresented sequences:*

    The top overrepresented sequences were:
    CTAAAATCCCTCTTCGGATCCACAGTCAACCGCCCTGAACACATCCTGCA	4	246420	0.0314%
    CGAAGCTAAAATCCCTCTTCGGATCCACAGTCAACCGCCCTGAACACATC	3	169383	0.0216%
    AGATCGGAAGAGCACACGTCTGAACTCCAGTCACCAGATCATCTCGTATG	3	578466	0.0736%
    AGATCGGAAGAGCACACGTCTGAACTCCAGTCACCGATGTATCTCGTATG	3	228204	0.0290%
    GATCGGAAGAGCACACGTCTGAACTCCAGTCACCAGATCATCTCGTATGC	2	200072	0.0255%
    GGAAAACCCATTCTTCTCCCAGCCGGGTGCCCCAATACTTCAGTGCATGG	1	37188	0.0047%
    Most of these are known illumina sequence adapters and the rest are human sequences.

*Adapter content:*

    Most of the samples have illumina sequence adapters or poly-A tails after bp 92 and some even at bp 67

If necessary, use `cutadapt` to trim and filter your reads. Provide a justification for the `cutadapt` parameters you used:



In [ ]:
# !mkdir fastq_placenta/placenta_qc
!fastqc -t 16 fastq_placenta/*.fastq.gz -o fastq_placenta/placenta_qc

In [9]:
!cutadapt -a AATGATACGGCGACCACCGAGATCTACACTCTTTCCCTACACGACGCTCTTCCGATCT -u 10 -q 28 -o ./fastq_placenta/trimmed_placenta/ERR4231018.fastq.gz fast

In [25]:
# %mkdir fastq_placenta/trimmed_placenta
!for fastq in fastq_placenta/*.fastq.gz; do \
    base_name=$(basename "$fastq" .fastq.gz); \
    cutadapt -a AATGATACGGCGACCACCGAGATCTACACTCTTTCCCTACACGACGCTCTTCCGATCT -u 10 -q 28 -o ./fastq_placenta/trimmed/$base_name.fastq.gz $fastq; \
done

/data/users/mccallke0364/BIOL343/5_fastq/fastq_placenta
ERR4231010.fastq.gz  ERR4231016.fastq.gz  ERR4231022.fastq.gz
ERR4231011.fastq.gz  ERR4231017.fastq.gz  ERR4231023.fastq.gz
ERR4231012.fastq.gz  ERR4231018.fastq.gz  placenta_qc
ERR4231013.fastq.gz  ERR4231019.fastq.gz  trimmed_placenta
ERR4231014.fastq.gz  ERR4231020.fastq.gz
ERR4231015.fastq.gz  ERR4231021.fastq.gz
This is cutadapt 4.9 with Python 3.12.3
Command line parameters: -u 10 -q 28 -o ./trimmed_placenta/ERR4231010.fastq.gz ./ERR4231010.fastq.gz
Processing single-end reads on 1 core ...

No reads processed!
This is cutadapt 4.9 with Python 3.12.3
Command line parameters: -u 10 -q 28 -o ./trimmed_placenta/ERR4231011.fastq.gz ./ERR4231011.fastq.gz
Processing single-end reads on 1 core ...

No reads processed!
This is cutadapt 4.9 with Python 3.12.3
Command line parameters: -u 10 -q 28 -o ./trimmed_placenta/ERR4231012.fastq.gz ./ERR4231012.fastq.gz
Processing single-end reads on 1 core ...

No reads processed!
This is cutad

Run QC on the trimmed/filtered reads and generate an aggregated report

In [10]:
!multiqc fastq_placenta/.


  /// ]8;id=893344;https://multiqc.info\MultiQC]8;;\ 🔍 | v1.17

|           multiqc | Search path : /data/users/mccallke0364/BIOL343/5_fastq/fastq_placenta
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 42/42  0m  
|            fastqc | Found 14 reports
|           multiqc | Existing reports found, adding suffix to filenames. Use '--force' to overwrite.
|           multiqc | Report      : multiqc_report_1.html
|           multiqc | Data        : multiqc_data_1
|           multiqc | MultiQC complete


Report your observations on which QC metrics improved (or got worse):



## Snakemake Pipeline

When running bioinformatics studies, it is wise to use notebooks (such as this one) to tie together code, decisions, and data visualization. The above task is something you might generate and report to a client or supervisor who might be interested in sequencing QC and what kind of pre-processing decisions you made prior to alignment. In addition, the preprocessing should be readily reproducible by all relevant stakeholders, so you should supplement notebooks with pipeline scripts. Throughout the semester, we will be creating snakemake pipelines to reproduce the published analysis that you've selected. Follow the below steps to begin producing this pipeline:

1. Make a new `Snakefile` and `config.yaml` in `5_fastq`. 
2. In the `Snakefile`, produce the following rules:  
    a. `fetch_fastq` - use `parallel-fastq-dump` to fetch every FASTQ file listed in `config.yaml`.  
    b. `fastq_qc` - use `fastqc` to run QC on the raw FASTQ files.  
    c. `trim_filter` - use `cutadapt` to trim and/or filter the FASTQ reads using parameters justified in this notebook. ***Note:*** if you choose to trim/filter, it should be done on *all* of the FASTQ files; do not cherry pick.  
    d. `trim_qc` - use `fastqc` to run QC on the trimmed FASTQ files.  
    e. `report` - use `multiqc` to aggregate all QC data into a single report.

This `Snakefile` should not use any `wrappers` because we already have the relevant software installed in the `biol343` conda environment. The pipeline should run to completion when any instructor or classmate runs `snakemake --use-conda`. You may use the documentation found [here](https://multiqc.info/docs/usage/pipelines/#snakemake) or any other online documentation you may find. You may not use any AI tools to complete this homework.